# 2022 Flatiron Machine Learning x Science Summer School

## Step 6: Train DSN with $L_1$ regularization on latent features using GhostAdam

Based on [SR3](https://arxiv.org/abs/1807.05411)

This is an approach to minimize prediction accuracy and regularization separately

With the DSN, do we still have $a_1$ and $a_2$ combined?

Let's explore this with a simple sweep

Stuff is not working

ghost_loss_backward:

* get_param_differences_and_scales:

    * get param_groups: dict per live/ghost
    
    * betas (Tuple[float, float], optional): coefficients used for computing running averages of gradient and its square default: (0.9, 0.999))
    
    * go through parameters
    
    * 
    
    
    
Sanity check: set requires grad to 0 

Do we make sure that unregularized parameters are not effected by our trick?




We need to make sure that ghost parameters change with live parameters:
    Set differences to 0 and set p.ghost to p.live?
    
necessary?
all_p.append(p.live.sum() * 0.0 + p.ghost.sum() * 0.0)


is hack slowing us down?

ghost L1 activation:
13:28<00:00, 12.36it/s, train_loss=2.09e-06, val_loss=1.20e-03

vs. regular L1 loss:
09:43<00:00, 17.15it/s, train_loss=3.34e-04, val_loss=1.83e-03

vs. no ghost L1 activation:
09:08<00:00, 18.23it/s, train_loss=1.90e-03, val_loss=1.92e-03

vs. ghost L1 activation via 0 loss:
13:12<00:00, 12.63it/s, train_loss=2.09e-06, val_loss=1.20e-03

Error:

python 3.7:
reset_parameters()
    self.model.parameters()
        self.
        model.: return GhostTuple
        parameters(): GhostTuple: __getattr__ (name = parameters):
            return GhostTuple(
                live=getattr(self.live, name),
                ghost=getattr(self.ghost, name),
            )
            __getitem__(index = 0):
                 return GhostTuple(live=self.live[index], ghost=self.ghost[index])


python 3.7:
reset_parameters()
    self.model.parameters()
        self.
        model.: return GhostTuple
        parameters(): GhostTuple: __getattr__ (name = parameters):
            return GhostTuple(
                live=getattr(self.live, name),
                ghost=getattr(self.ghost, name),
            )
            
            just works
            
            
https://docs.python.org/3/whatsnew/3.8.html
Sped-up field lookups in collections.namedtuple(). They are now more than two times faster, making them the fastest form of instance variable lookup in Python. (Contributed by Raymond Hettinger, Pablo Galindo, and Joe Jevnik, Serhiy Storchaka in bpo-32492.)

namedtuple isn't a class, as you note; it's a function. But it's a function that returns a class. Thus, you can use the result of the namedtuple call as a parent class.

The problem with GhostTuples (GT) is that (in Python 3.7) GT1.live is processed as __getitem__(GT1, 0).

So do we want __getitem__(GT1, 0) generally to return GT1.live or do we want GT2(live=GT1.live[0], ghost=GT1.ghost[1])?




https://stackoverflow.com/questions/6738087/why-doesnt-python-have-a-hybrid-getattr-getitem-built-in
I've given a fair bit of thought to this question, and I think the answer is very simple. When you create a container type, it's very important to distinguish between attributes and items. Any reasonably well-developed container type will have a number of attributes -- often though not always methods -- that enable it to manage its contents in graceful ways. So for example, a dict has items, values, keys, iterkeys and so on. These attributes are all accessed using . notation. Items, on the other hand, are accessed using [] notation. So there can be no collisions.

What happens when you enable item access using . notation? Suddenly you have overlapping namespaces. How do you handle collisions now? If you subclass a dict and give it this functionality, either you can't use keys like items as a rule, or you have to create some kind of namespace hierarchy. The first option creates a rule that is onerous, hard to follow, and hard to enforce. The second option creates an annoying amount of complexity, without fully resolving the collision problem, since you still have to have an alternative interface to specify whether you want items the item or items the attribute.

Now, for certain kinds of very primitive types, this is acceptable. That's probably why there's namedtuple in the standard library, for example. (But note that namedtuple is subject to these very problems, which is probably why it was implemented as a factory function (prevents inheritance) and uses weird, private method names like _asdict.)

---

Do we even want to create new GhostTuples all the time?


py37: ghost_class
01:36<00:00, 10.38it/s, train_loss=2.36e-04, val_loss=3.72e-03

py38: ghost_class
01:23<00:00, 11.92it/s, train_loss=2.36e-04, val_loss=3.72e-03

py38: ghost_tuple
01:25<00:00, 11.67it/s, train_loss=2.36e-04, val_loss=3.72e-03

## 6.1 L1

In [1]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import matplotlib.pyplot as plt
import joblib

import torch
import wandb

from srnet import SRNet, SRData
import srnet_utils as ut

In [2]:
# set wandb project
wandb_project = "61-l1-gc-study-F00"

In [3]:
# hyperparams = {
#     "arch": {
#         "in_size": train_data.in_data.shape[1],
#         "out_size": train_data.target_data.shape[1],
#         "hid_num": (2,0),
#         "hid_size": 32, 
#         "hid_type": ("DSN", "MLP"),
#         "lat_size": 16,
#         },
#     "epochs": 10000,
#     "runtime": None,
#     "batch_size": 64,
#     "lr": 1e-4,
#     "wd": 1e-4,
#     "l1": 0.0,
#     "a1": 0.0,
#     "a2": 0.0,
#     "gc": 0.0,
#     "shuffle": True,
# }

In [4]:
# define hyperparameter study
hp_study = {
    "method": "grid", # random, bayesian
    #"metric": {
    #    "name": "val_loss",
    #    "goal": "minimize",
    #},
    "parameters": {
        "l1": {
            "values": [1e-4, 1e-3, 1e-2]
        },
        "gc": {
            "values": [1e-5, 1e-4, 1e-3]
        }
    }
}

In [5]:
# create sweep
sweep_id = wandb.sweep(hp_study, project=wandb_project)

Create sweep with ID: ukt7pwp4
Sweep URL: https://wandb.ai/fabxy/61-l1-gc-study-F00/sweeps/ukt7pwp4


In [ ]:
# download data from wandb
file_ext = ".pkl"

api = wandb.Api()

runs = api.runs(wandb_project)
for run in runs:
    for f in run.files():
        if f.name[-len(file_ext):] == file_ext and not os.path.isfile(f.name):
            print(f"Downloading {os.path.basename(f.name)}.")
            run.file(f.name).download()

## 6.2 DSN

In [6]:
# set wandb project
wandb_project = "62-a1-a2-gc-study-F00"

In [7]:
# define hyperparameters
# hyperparams = {
#     "arch": {
#         "in_size": train_data.in_data.shape[1],
#         "out_size": train_data.target_data.shape[1],
#         "hid_num": (2,0),
#         "hid_size": 32, 
#         "hid_type": ("DSN", "MLP"),
#         "lat_size": 16,
#         },
#     "epochs": 10000,
#     "runtime": None,
#     "batch_size": 64,
#     "lr": 1e-4,
#     "wd": 1e-4,
#     "l1": 0.0,
#     "a1": 0.0,
#     "a2": 0.0,
#     "gc": 0.0,
#     "shuffle": True,
# }

In [8]:
# define hyperparameter study
hp_study = {
    "method": "grid", # random, bayesian
    #"metric": {
    #    "name": "val_loss",
    #    "goal": "minimize",
    #},
    "parameters": {
        "a1": {
            "values": [1e-5, 1e-3]
        },
        "a2": {
            "values": [1e-5, 1e-3]
        },
        "gc": {
            "values": [1e-5, 1e-4, 1e-3]
        }
    }
}

In [9]:
# create sweep
sweep_id = wandb.sweep(hp_study, project=wandb_project)

Create sweep with ID: h8ngmjg3
Sweep URL: https://wandb.ai/fabxy/62-a1-a2-gc-study-F00/sweeps/h8ngmjg3


In [ ]:
# download data from wandb
file_ext = ".pkl"

api = wandb.Api()

runs = api.runs(wandb_project)
for run in runs:
    for f in run.files():
        if f.name[-len(file_ext):] == file_ext and not os.path.isfile(f.name):
            print(f"Downloading {os.path.basename(f.name)}.")
            run.file(f.name).download()